In [ ]:
import os
from tensorflow.python.profiler import profiler_client

tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
print(profiler_client.monitor(tpu_profile_service_address, 100, 2))

  Timestamp: 00:57:50
  TPU type: TPU v2
  Utilization of TPU Matrix Units (higher is better): 0.000%




In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
  raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

Tensorflow version 2.8.2
Running on TPU  ['10.64.34.74:8470']


In [ ]:
import re
from tqdm import tqdm
from tqdm.auto import trange
import time
import os

from typing import List, Dict, Union, Tuple, NoReturn

import pandas as pd
from pandas.core.series import Series
import json

import numpy as np
import string

import nltk    
from nltk import tokenize    
nltk.download('punkt')   

from sklearn.model_selection import train_test_split

import keras
from keras.models import load_model, model_from_json, Sequential
from keras.layers import Dense,Conv1D,MaxPooling1D, GlobalMaxPooling1D, GlobalAveragePooling1D
from keras.layers import Flatten, LSTM, Bidirectional, Dropout, concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras import Input, Model
from keras.callbacks import ModelCheckpoint

import tensorflow as tf

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
GDRIVE_PATH:str = '/content/drive/MyDrive'
DATASET_ROOT_PATH: str = os.path.join(GDRIVE_PATH, 'dataset', 'projeto-final')
DATASET_IMPRENSA: str = os.path.join(DATASET_ROOT_PATH, 'imprensa')
DATASET_MENCOES: str = os.path.join(DATASET_ROOT_PATH, 'mencoes')
CONTEUDO_INDISPONIVEL: str = 'CONTEUDO_INDISPONIVEL'
STOP_WORDS_FILE: str = os.path.join(DATASET_ROOT_PATH, 'stop-words.txt')

EMBEDDING_GLOVE_50 : str = os.path.join(GDRIVE_PATH, 'model', 'embedding','glove_s50.txt')
EMBEDDING_WORD2VEC_300: str = os.path.join(GDRIVE_PATH, 'model', 'embedding','cbow_s300.txt')
EMBEDDING_WORD2VEC_600: str = os.path.join(GDRIVE_PATH, 'model', 'embedding','cbow_s600.txt')
EMBEDDING_WORD2VEC_1000: str = os.path.join(GDRIVE_PATH, 'model', 'embedding','cbow_s1000.txt')
EMBEDDING_GLOVE_1000: str = os.path.join(GDRIVE_PATH, 'model', 'embedding','glove_s1000.txt')

LABEL_NAMES: List[str] = ['Negativa', 'Neutra', 'Positiva']
LABEL_DICT_CONV_MENCOES : Dict[int,str] = {-5:'Negativa', 0:'Neutra', 5:'Positiva'}

MAX_SENTENCE_LENGTH: int = 1000 
MODEL_PATH: str = os.path.join(GDRIVE_PATH, 'model', 'projeto-final')

# palavras comumente incorretas
SUBSTITUICOES_COMUNS_PALAVRAS_INCORRETAS: Dict[str,str] = {
    'covid-00': 'coronavírus',  'privatizacao':  'privatização', 'leilao':  'leilão',
    'inflacao':  'inflação', 'bilhao':  'bilhão', 'concessoes': 'concessões',
    'aprovacao': 'aprovação', 'covid': 'coronavírus', 'desestatizacao': 'desestatização',
    'governanca': 'governança', 'atuacao': 'atuação', 'emissoes': 'emissões', 
    'manutencao': 'manutenção', 'licitacao': 'licitação', 'protecao': 'proteção',
    'emissao': 'emissão', 'contratacao': 'contratação', 'aquisicao': 'aquisição',
    'arrecadacao': 'arrecadação', 'votacao': 'votação', 'ampliacao': 'ampliação',
    'negociacao': 'negociação', 'vacinacao': 'vacinação', 'inadimplencia': 'inadimplência', 
    'poupanca': 'poupança', 'realizacao': 'realização', 'suspensao': 'suspensão', 
    'preservacao': 'preservação', 'estruturacao': 'estruturação', 'fiscalizacao': 'fiscalização',
    'capitalizacao': 'capitalização', 'conservacao': 'conservação', 'prestacao': 'prestação',
    'cobranca': 'cobrança', 'transicao': 'transição', 'remuneracao': 'remuneração',
    'liberacao': 'liberação', 'discussoes': 'discussões', 'universalizacao': 'universalização',
    'aculpa': 'culpa', 'deverao': 'deverão', 'elaboracao': 'elaboração',
    'contribuicao': 'contribuição', 'mineracao': 'mineração', 'adesao': 'adesão',
    'modernizacao': 'modernização','regulacao': 'regulação', 'projecao': 'projeção',
    'regulatorio': 'regulatório', 'centrao': 'centrão', 'avancos': 'avanços',
    'climatica': 'climática', 'variacao': 'variação', 'implantacao': 'implantação',
    'implementacao': 'implementação', 'projecoes': 'projeções', 'senadorhumberto': 'senador',
    'trilhao': 'trilhão', 'reeleicao':  'reeleição', 'restricoes': 'restrições',
    'elevacao': 'elevação', 'percepcao': 'percepção', 'importacao': 'importação',
    'exportacao':  'exportação', 'valorizacao':  'valorização', 'licitacoes': 'licitações', 
    'adocao': 'adoção', 'trilhoes': 'trilhões', 'sustentaveis': 'sustentáveis',
    'aviacao': 'aviação', 'pregao': 'pregão'
}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def load_stopwords():
    """
    This function loads a stopword list from the *path* file and returns a 
    set of words. Lines begining by '#' are ignored.
    """

    # Set of stopwords
    stopwords = set([])

    # For each line in the file
    with open(STOP_WORDS_FILE, 'r', encoding='utf-8') as f:
        for line in f:
            if not re.search('^#', line) and len(line.strip()) > 0:
                stopwords.add(line.strip().lower())

    # inclusão dos tokens gerados incorretamente pelo word tokenize
    stopwords.add("``")
    stopwords.add("''")
    # Return the set of stopwords
    return stopwords

In [ ]:
def pre_processar(text: str) -> str:
    re_remove_brackets = re.compile(r'\{.*\}')
    re_remove_html = re.compile(r'<(\/|\\)?.+?>', re.UNICODE)
    re_transform_numbers = re.compile(r'\d', re.UNICODE)
    re_transform_emails = re.compile(r'[^\s]+@[^\s]+', re.UNICODE)
    re_transform_url = re.compile(r'(http|https)://[^\s]+', re.UNICODE)
    # Different quotes are used.
    re_quotes_1 = re.compile(r"(?u)(^|\W)[‘’′`']", re.UNICODE)
    re_quotes_2 = re.compile(r"(?u)[‘’`′'](\W|$)", re.UNICODE)
    re_quotes_3 = re.compile(r'(?u)[‘’`′“”]', re.UNICODE)
    re_dots = re.compile(r'(?<!\.)\.\.(?!\.)', re.UNICODE)
    re_punctuation = re.compile(r'([,";:]){2},', re.UNICODE)
    re_hiphen = re.compile(r' -(?=[^\W\d_])', re.UNICODE)
    re_tree_dots = re.compile(u'…', re.UNICODE)
    # Differents punctuation patterns are used.
   # re_punkts = re.compile(r'(\w+)([%s])([ %s])' %
    #                    (punctuations, punctuations), re.UNICODE)
    #re_punkts_b = re.compile(r'([ %s])([%s])(\w+)' %
    #                        (punctuations, punctuations), re.UNICODE)
    #re_punkts_c = re.compile(r'(\w+)([%s])$' % (punctuations), re.UNICODE)
    re_changehyphen = re.compile(u'–')
    re_doublequotes_1 = re.compile(r'(\"\")')
    re_doublequotes_2 = re.compile(r'(\'\')')
    re_trim = re.compile(r' +', re.UNICODE)
    
    """Apply all regex above to a given string."""
    text = text.lower()
    text = text.replace('\xa0', ' ')
    text = re_tree_dots.sub('...', text)
    text = re.sub('\.\.\.', '', text)
    text = re_remove_brackets.sub('', text)
    text = re_changehyphen.sub('-', text)
    text = re_remove_html.sub(' ', text)
    text = re_transform_numbers.sub('0', text)
    text = re_transform_url.sub('URL', text)
    text = re_transform_emails.sub('EMAIL', text)
    text = re_quotes_1.sub(r'\1"', text)
    text = re_quotes_2.sub(r'"\1', text)
    text = re_quotes_3.sub('"', text)
    text = re.sub('"', '', text)
    text = re_dots.sub('.', text)
    text = re_punctuation.sub(r'\1', text)
    text = re_hiphen.sub(' - ', text)
    #text = re_punkts.sub(r'\1 \2 \3', text)
    #text = re_punkts_b.sub(r'\1 \2 \3', text)
    #text = re_punkts_c.sub(r'\1 \2', text)
    text = re_doublequotes_1.sub('\"', text)
    text = re_doublequotes_2.sub('\'', text)
    text = re_trim.sub(' ', text)
    
    return text.strip()


In [ ]:
def tokenizar(lista_texto: List[str]) -> List[List[str]]:
    lista_tokens_clear: List[List[str]] = []
    stop_words = load_stopwords()

    for texto in tqdm(lista_texto, 'Tokenizando... '):
        texto = pre_processar(texto)
        lista_tokens = tokenize.word_tokenize(texto, language='portuguese')
        lista_tokens = [token.lower() for token in lista_tokens if token.lower() not in stop_words and token not in string.punctuation]
        lista_tokens_clear.append(lista_tokens)
        
    return lista_tokens_clear

In [ ]:
def read_embedding(file_name, dim_size, debug: bool = False):
    with open(file_name,'r', encoding="utf8") as f:
        vocab:Dict[str, int] = {} 
        word_vector: List[np.nparray] = []
        pos = 0
        for line_number, line in enumerate(f):
            line_ = line.strip() 
            line_word_vec = line_.split()
            
            if (len(line_word_vec) == dim_size + 1):
                if not line_word_vec[0] in vocab:
                    vocab[line_word_vec[0]] = pos
                    word_vector.append(np.array(line_word_vec[1:],dtype=float)) 
                    pos+=1
            else:
                if debug:
                    print(f'Linha {line_number} Ignorada', line_word_vec)
            
    if debug:
        print("Total palavras no embedding :", len(vocab))
    np_word_vector: np.nparray = np.stack(word_vector)
    return vocab, np_word_vector

In [ ]:
def trata_tokens_ausentes(lista_textos_tokenizados: List[List[str]], 
                          vocab: List[str], 
                          debug: bool = False) -> Tuple[List[str], Dict[str,int]] :
    '''
        Trata tokens ausentes retirando-os da lista. Pode-se tentar salvar o token com o paramentro de salvar

        lista_textos_tokenizados:
            Lista de lista de tokens extraídos de texto
        salvar_ausente:
            Tenta salvar o ausente buscando palavras próximas (utilizando proximidade de edição)
         debug:
            Logar informações sobre o processo

        Returns:
            List[str] - Tokens limpos para o modelo, ou seja, todos estão no embedding
            Dict[str,int] - Dicionário com palavras ausentes e suas respectivas quantidades
    '''

    dict_tokens_ausentes: Dict[str,int] = {}
    
    lista_textos_tokenizados_ajustada: List[List[str]] = []
    for lista_tokens in tqdm(lista_textos_tokenizados, 'Tratamento tokens ausentes...'):
        lista_token_ajustada: List[str] = []
        for token in lista_tokens:
            if token in vocab:
                lista_token_ajustada.append(token)
            else:
                if token in SUBSTITUICOES_COMUNS_PALAVRAS_INCORRETAS:
                    lista_token_ajustada.append(SUBSTITUICOES_COMUNS_PALAVRAS_INCORRETAS[token])
                else:
                    if token in dict_tokens_ausentes:
                        dict_tokens_ausentes[token] +=1
                    else:
                        dict_tokens_ausentes[token] = 1
        lista_textos_tokenizados_ajustada.append(lista_token_ajustada)

    if debug:
        sorted_ausentes = sorted(dict_tokens_ausentes, key=dict_tokens_ausentes.get, reverse=True)
        for i, r in enumerate(sorted_ausentes):
            print(r, dict_tokens_ausentes[r])
            if (i > 20):
                break

    return lista_textos_tokenizados_ajustada

In [ ]:
def converte_tokens_to_id(lista_sentencas_tokenizadas: List[List[str]], vocab: Dict[str,int], max_sentence_length: int) -> np.ndarray:
    conversao = np.zeros((len(lista_sentencas_tokenizadas), max_sentence_length), dtype='int32')
    
    for count_sentenca,sentenca in enumerate(lista_sentencas_tokenizadas):
        for count_token,token in enumerate(sentenca):
            if(count_token < max_sentence_length):
                conversao[count_sentenca,count_token] = vocab[token]
        
    return conversao


In [ ]:
vocab, word_vector = read_embedding(EMBEDDING_WORD2VEC_300, 300) 


In [ ]:
filepath = os.path.join(MODEL_PATH, "bndes_sentiment.hdf5")
model_load = tf.keras.models.load_model(filepath)


In [ ]:
teste: List[str] =[ 'Um amigo #Cubano está perguntando se o PT continuará investindo em #Cuba com dinheiro do #BNDES(do povo #brasileiro). Falei pra ele ficar tranquilo, pois #Lula fará trens, portos, aeroportos, e muito mais, em Cuba. Esse item do plano de governo do PT mostra bem isso, correto?',
                   'O Programa Emergencial de Acesso a Crédito, tem como objetivo possibilitar a ampliação do acesso ao crédito para MEIs, micro, pequenas e médias empresas, permitindo a manutenção do emprego e da renda.',
                   'Manda ele pegar o #Metro de Caracas pago pelo #BNDES  e levar pra  #BeloHorizonte. #PTNuncamais',
                   'Falei lá atrás que o #PTNuncaMais aparelhou junto com #Cuba, #Venezuela e mais 13 nações via #BNDES e me perdoem #patriotas não lembrar o nome da #Bolívia que foi sim ajudada pelo #bandido #comunista via #Petrobrás com 3 refinarias a custo zero para aquele país sul americano.',
                   'O Banco Nacional de Desenvolvimento Econômico e Social (#BNDES) adiou de 22 de agosto para o próximo dia 6 de setembro a realização de audiência pública para discutir a desestatização do Porto de Santos.']

In [ ]:
lista_tokens : List[List[str]] = tokenizar(teste)
lista_tokens = trata_tokens_ausentes(lista_tokens, vocab)
np_tokens: np.ndarray = converte_tokens_to_id(lista_tokens, vocab, MAX_SENTENCE_LENGTH )



Tratamento tokens ausentes...: 100%|██████████| 5/5 [00:00<00:00, 4765.17it/s]


In [ ]:
predict_logits = model_load.predict(np_tokens)

In [ ]:
predict = np.argmax(predict_logits, axis=1)

In [ ]:
for index, t in enumerate(teste):
    print(f' Sentimento: {LABEL_NAMES[predict[index]]} /n {t}')
    print(' ===============================================')

 Sentimento: Negativa /n Um amigo #Cubano está perguntando se o PT continuará investindo em #Cuba com dinheiro do #BNDES(do povo #brasileiro). Falei pra ele ficar tranquilo, pois #Lula fará trens, portos, aeroportos, e muito mais, em Cuba. Esse item do plano de governo do PT mostra bem isso, correto?
 Sentimento: Positiva /n O Programa Emergencial de Acesso a Crédito, tem como objetivo possibilitar a ampliação do acesso ao crédito para MEIs, micro, pequenas e médias empresas, permitindo a manutenção do emprego e da renda.
 Sentimento: Negativa /n Manda ele pegar o #Metro de Caracas pago pelo #BNDES  e levar pra  #BeloHorizonte. #PTNuncamais
 Sentimento: Negativa /n Falei lá atrás que o #PTNuncaMais aparelhou junto com #Cuba, #Venezuela e mais 13 nações via #BNDES e me perdoem #patriotas não lembrar o nome da #Bolívia que foi sim ajudada pelo #bandido #comunista via #Petrobrás com 3 refinarias a custo zero para aquele país sul americano.
 Sentimento: Neutra /n O Banco Nacional de Desenv